# Multi-head Latent Attention

Multi-head Latent Attention (MLA) is a variant of MHA that drastically reduce the memory footprint and compute cost of th KV cache in LLMs in inference.


Traditional MHA caches large K and V matrices for each token, which grows the cache size quadratically as the sequence length grows and becomes a major bottleneck for long contexts.

MLA addresses this with low-rank compression, projecting the input hidden state into a much smaller latent space. It introduces a down-projection layer that compresses the large K and V matrices into a single, much smaller latent representation (a "latent KV" matrix) and stores only this small latent representation in the KV cache. At attention computation, this latent matrix is then "up-projected" by specific per-head linear layers to reconstruct the K and V vectors in their respective head dimensions.

It was introduced in DeepSeek-v2 paper [DeepSeek-V2: A Strong, Economical, and Efficient Mixture-of-Experts Language Model](https://arxiv.org/abs/2405.04434), where in the ablation tests they found that MLA even perform better than the traditional MHA. MLA is also used in DeepSeek-v3 and DeepSeek R1.

MLA pairs especially well with KV-Cache at inference time by greatly reducing KV cache memory footprint. The inference memory efficiency makes MLA suitable for scenarios where inference speed is critical or memory is constraint, e.g.:
- long-context LLM (without hitting memory limits)
- edge and mobile devices
- efficient inference servers (serve faster and more users on a single GPU)

MLA often incorporates a "decoupled" RoPE. As the standard RoPE directly modifies K and V, in MLA, applying RoPE direcly on compressed K and V can be problematic or inefficient. 

## Code